In [1]:
from time import sleep

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F


In [2]:
spark = SparkSession. \
    builder. \
    appName("Data Sources"). \
    master("local"). \
    config("spark.jars", "../jars/postgresql-42.2.19.jar"). \
    config("spark.sql.legacy.timeParserPolicy", "LEGACY"). \
    config("spark.sql.autoBroadcastJoinThreshold", -1). \
    getOrCreate()

In [5]:
movies_df = spark.read.json("data/movies")

In [ ]:
# aggregations

In [4]:
# counting
all_movies_count_df = movies_df.selectExpr("count(Major_Genre)")
print("SIMPLE COUNT")
all_movies_count_df.show()
all_movies_count_df.explain()

SIMPLE COUNT
+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(Major_Genre#13)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#46]
      +- HashAggregate(keys=[], functions=[partial_count(Major_Genre#13)])
         +- FileScan json [Major_Genre#13] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [6]:
 # null not included
genres_count_df = movies_df.select(count(col("Major_Genre")))
print("SIMPLE COUNT Genre")
genres_count_df.show()
genres_count_df.explain()


SIMPLE COUNT Genre
+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(Major_Genre#65)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#94]
      +- HashAggregate(keys=[], functions=[partial_count(Major_Genre#65)])
         +- FileScan json [Major_Genre#65] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [7]:
 # null not included
genres_count_df = movies_df.select(count(col("Major_Genre")))
print("SIMPLE COUNT Genre")
genres_count_df.show()
genres_count_df.explain()

SIMPLE COUNT Genre
+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(Major_Genre#65)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#139]
      +- HashAggregate(keys=[], functions=[partial_count(Major_Genre#65)])
         +- FileScan json [Major_Genre#65] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [8]:
# null included
genres_count_df_v2 = movies_df.selectExpr("count(*)")
print("SIMPLE COUNT Genre Expr")
genres_count_df_v2.show()

SIMPLE COUNT Genre Expr
+--------+
|count(1)|
+--------+
|    3201|
+--------+



In [9]:
# null included

genres_count_df_v2 = movies_df.selectExpr("count(*)")
print("SIMPLE COUNT Genre Expr")
genres_count_df_v2.show()


SIMPLE COUNT Genre Expr
+--------+
|count(1)|
+--------+
|    3201|
+--------+



In [11]:
 # null included  .count() is Action
genres_count_number = movies_df.select("Major_Genre").count()
print("SIMPLE COUNT Genre wit select")
print(genres_count_number)

SIMPLE COUNT Genre wit select
3201


In [13]:
# count distinct 
unique_genres_df = movies_df.select(F.countDistinct(F.col("Major_Genre")))
print("COUNT DISTINCT Genre wit select")
unique_genres_df.show()

COUNT DISTINCT Genre wit select
+---------------------------+
|count(DISTINCT Major_Genre)|
+---------------------------+
|                         12|
+---------------------------+



In [14]:
unique_genres_df_v2 = movies_df.selectExpr("count(DISTINCT Major_Genre)")
print("COUNT DISTINCT Genre with Expression")
unique_genres_df_v2.show()


COUNT DISTINCT Genre with Expression
+---------------------------+
|count(DISTINCT Major_Genre)|
+---------------------------+
|                         12|
+---------------------------+



# math aggregations

In [15]:
# min/max
max_rating_df = movies_df.select(max(col("IMDB_Rating")).alias("max value of IMDB Rating"))
print("max IMDB_Rating with functions")
max_rating_df.show()

max_rating_df_v2 = movies_df.selectExpr("min(IMDB_Rating)").alias("mix value of IMDB Rating")
print("max IMDB_Rating with Expression")
max_rating_df_v2.show()


max IMDB_Rating with functions
+------------------------+
|max value of IMDB Rating|
+------------------------+
|                     9.2|
+------------------------+

max IMDB_Rating with Expression
+----------------+
|min(IMDB_Rating)|
+----------------+
|             1.4|
+----------------+



In [16]:
# sum values in a column
us_industry_total_df = movies_df.select(sum(col("US_Gross")))
print("sum US_Gross")
us_industry_total_df.show()

us_industry_total_df_v2 = movies_df.selectExpr("sum(US_Gross)")
print("sum US_Gross Expr")
us_industry_total_df_v2.show()


sum US_Gross
+-------------+
|sum(US_Gross)|
+-------------+
| 140542660013|
+-------------+

sum US_Gross Expr
+-------------+
|sum(US_Gross)|
+-------------+
| 140542660013|
+-------------+



In [17]:
# avg
avg_rt_rating_df = movies_df.select(avg(col("Rotten_Tomatoes_Rating")))
print("avg Rotten_Tomatoes_Rating")
avg_rt_rating_df.show()


avg Rotten_Tomatoes_Rating
+---------------------------+
|avg(Rotten_Tomatoes_Rating)|
+---------------------------+
|          54.33692373976734|
+---------------------------+



In [18]:
# mean/standard dev
rt_stats_df = movies_df.agg(
    mean(col("Rotten_Tomatoes_Rating")),
    stddev(col("Rotten_Tomatoes_Rating"))
)
print("mean/standard dev")
rt_stats_df.show()


mean/standard dev
+---------------------------+-----------------------------------+
|avg(Rotten_Tomatoes_Rating)|stddev_samp(Rotten_Tomatoes_Rating)|
+---------------------------+-----------------------------------+
|          54.33692373976734|                  28.07659263787602|
+---------------------------+-----------------------------------+



# Grouping

In [19]:
# nulls are also considered
count_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    count()
print("count group by dev")
count_by_genre_df.show()
count_by_genre_df.explain()

count group by dev
+-------------------+-----+
|        Major_Genre|count|
+-------------------+-----+
|          Adventure|  274|
|               null|  275|
|              Drama|  789|
|        Documentary|   43|
|       Black Comedy|   36|
|  Thriller/Suspense|  239|
|            Musical|   53|
|    Romantic Comedy|  137|
|Concert/Performance|    5|
|             Horror|  219|
|            Western|   36|
|             Comedy|  675|
|             Action|  420|
+-------------------+-----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Major_Genre#65], functions=[count(1)])
   +- Exchange hashpartitioning(Major_Genre#65, 200), ENSURE_REQUIREMENTS, [id=#663]
      +- HashAggregate(keys=[Major_Genre#65], functions=[partial_count(1)])
         +- FileScan json [Major_Genre#65] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: str

In [21]:
# Grouping and counting
avg_rating_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    avg("IMDB_Rating")
print("avg group by dev")
avg_rating_by_genre_df.show()
avg_rating_by_genre_df.explain()


avg group by dev
+-------------------+------------------+
|        Major_Genre|  avg(IMDB_Rating)|
+-------------------+------------------+
|          Adventure| 6.345019920318729|
|               null|  6.50082644628099|
|              Drama| 6.773441734417339|
|        Documentary| 6.997297297297298|
|       Black Comedy|6.8187500000000005|
|  Thriller/Suspense| 6.360944206008582|
|            Musical|             6.448|
|    Romantic Comedy| 5.873076923076922|
|Concert/Performance|             6.325|
|             Horror|5.6760765550239185|
|            Western| 6.842857142857142|
|             Comedy| 5.853858267716529|
|             Action| 6.114795918367349|
+-------------------+------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Major_Genre#65], functions=[avg(IMDB_Rating#62)])
   +- Exchange hashpartitioning(Major_Genre#65, 200), ENSURE_REQUIREMENTS, [id=#837]
      +- HashAggregate(keys=[Major_Genre#65], functions=[partial_avg(IM

In [22]:
# multiple aggregations
aggregations_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    agg(
    # use strings here for column names
    count("*").alias("N_Movies"),
    avg("IMDB_Rating").alias("Avg_Rating")
)
aggregations_by_genre_df.show()


+-------------------+--------+------------------+
|        Major_Genre|N_Movies|        Avg_Rating|
+-------------------+--------+------------------+
|          Adventure|     274| 6.345019920318729|
|               null|     275|  6.50082644628099|
|              Drama|     789| 6.773441734417339|
|        Documentary|      43| 6.997297297297298|
|       Black Comedy|      36|6.8187500000000005|
|  Thriller/Suspense|     239| 6.360944206008582|
|            Musical|      53|             6.448|
|    Romantic Comedy|     137| 5.873076923076922|
|Concert/Performance|       5|             6.325|
|             Horror|     219|5.6760765550239185|
|            Western|      36| 6.842857142857142|
|             Comedy|     675| 5.853858267716529|
|             Action|     420| 6.114795918367349|
+-------------------+--------+------------------+



In [24]:
# sorting
# TODO slide how to work rangepartitioning in Spark
best_movies_df = movies_df.orderBy(col("IMDB_Rating").desc())
best_movies_df.show()
best_movies_df.explain()

# sorting works for numerical, strings (lexicographic), dates


+--------------------+--------------------+--------------------+-----------+----------+-----------+-----------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|            Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|      Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+--------------------+--------------------+-----------+----------+-----------+-----------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|  Historical Fiction|Francis Ford Coppola|  Paramount Pictures|        9.2|    411088|       null|             null|          7000000|   15-Mar-72|                   100|            null|  

In [25]:
# put nulls first or last
proper_worst_movies_df = movies_df.orderBy(col("IMDB_Rating").asc_nulls_last())
proper_worst_movies_df.show()

+--------------------+------------------+------------------+-----------+----------+-----------+---------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+--------+---------------+
|       Creative_Type|          Director|       Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|    Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+------------------+------------------+-----------+----------+-----------+---------------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+--------+---------------+
|        Kids Fiction|              null|     Sony Pictures|        1.4|     10886|         PG|         Comedy|         20000000|   27-Aug-04|                  null|            null| Original Screenplay|Super Ba

Exercises

1. Sum up ALL the profits of ALL the movies in the dataset

2. Count how many distinct directors we have

3. Show the mean, stddev for US gross revenue

4. Compute the average IMDB rating, average US gross PER DIRECTOR

5. Show the average difference between IMDB rating and Rotten Tomatoes rating


# Joins

In [28]:
guitars_df = spark.read.json("data/guitars")
guitar_players_df = spark.read.json("data/guitarPlayers")
bands_df = spark.read.json("data/bands")


In [29]:
# inner joins = all rows from the "left" and "right" DF for which the condition is true
join_condition = guitar_players_df.band == bands_df.id
guitarists_bands_df = guitar_players_df.join(bands_df, join_condition, "inner")

guitarists_bands_df.show()
guitarists_bands_df.explain()


# How to work hash partitioning and join

#                     ^^ "left" DF            ^^ "right" DF 
# node 1 => 1|    [1]|  1| Angus Young|     Sydney|
# node 2 => 1|       AC/DC|1973|
# node 3 => 1|    [1]|  1| Angus Young|     Sydney| 1|       AC/DC|1973|
# node 4 => 2, 4
# node 5 => 11, 12
#


+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [band#837L], [id#853L], Inner
   :- Sort [band#837L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(band#837L, 200), ENSURE_REQUIREMENTS, [id=#1081]
   :     +- Filter isnotnull(band#837L)
   :        +- FileScan json [band#837L,guitars#838,id#839L,name#840] Batched: false, DataFilters: [isnotnull(band#837L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/guitarPlayers], PartitionFilters: [], PushedFilters: [

In [30]:
# differentiate the "name" column - use the reference from the original DF
guitarists_bands_upper_df = guitarists_bands_df.select(upper(bands_df.name))
guitarists_bands_upper_df.show()
guitarists_bands_upper_df.explain()


+------------+
| upper(name)|
+------------+
|LED ZEPPELIN|
|       AC/DC|
|   METALLICA|
+------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [upper(name#854) AS upper(name)#917]
   +- SortMergeJoin [band#837L], [id#853L], Inner
      :- Sort [band#837L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(band#837L, 200), ENSURE_REQUIREMENTS, [id=#1241]
      :     +- Filter isnotnull(band#837L)
      :        +- FileScan json [band#837L] Batched: false, DataFilters: [isnotnull(band#837L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/guitarPlayers], PartitionFilters: [], PushedFilters: [IsNotNull(band)], ReadSchema: struct<band:bigint>
      +- Sort [id#853L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(id#853L, 200), ENSURE_REQUIREMENTS, [id=#1242]
            +- Filter isnotnull(id#853L)
               +- FileScan json [id#853L,name#854] Batched: false, DataFilters: [isnotnull(id#853

In [31]:
# left outer = everything in the inner join + all the rows in the LEFT DF that were not matched (with nulls in the cols for the right DF)
guitar_players_df.join(bands_df, join_condition, "left_outer").show()


+----+-------+---+------------+-----------+----+------------+----+
|band|guitars| id|        name|   hometown|  id|        name|year|
+----+-------+---+------------+-----------+----+------------+----+
|   0|    [0]|  0|  Jimmy Page|     London|   0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     Sydney|   1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|       null|null|        null|null|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|   3|   Metallica|1981|
+----+-------+---+------------+-----------+----+------------+----+



In [32]:
# right outer = everything in the inner join + all the rows in the RIGHT DF that were not matched (with nulls in the cols for the left DF)
guitar_players_df.join(bands_df, join_condition, "right_outer").show()

+----+-------+----+------------+-----------+---+------------+----+
|band|guitars|  id|        name|   hometown| id|        name|year|
+----+-------+----+------------+-----------+---+------------+----+
|   0|    [0]|   0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   1|    [1]|   1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   3|    [3]|   3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
|null|   null|null|        null|  Liverpool|  4| The Beatles|1960|
+----+-------+----+------------+-----------+---+------------+----+



In [33]:
# full outer join = everythin in the inner join + all the rows in BOTH DFs that were not matched (with nulls in the other DF's cols)
guitar_players_df.join(bands_df, join_condition, "outer").show()

+----+-------+----+------------+-----------+----+------------+----+
|band|guitars|  id|        name|   hometown|  id|        name|year|
+----+-------+----+------------+-----------+----+------------+----+
|   0|    [0]|   0|  Jimmy Page|     London|   0|Led Zeppelin|1968|
|   1|    [1]|   1| Angus Young|     Sydney|   1|       AC/DC|1973|
|   2| [1, 5]|   2|Eric Clapton|       null|null|        null|null|
|   3|    [3]|   3|Kirk Hammett|Los Angeles|   3|   Metallica|1981|
|null|   null|null|        null|  Liverpool|   4| The Beatles|1960|
+----+-------+----+------------+-----------+----+------------+----+



In [34]:
# join on a single column
guitar_players_df.join(bands_df, "id").show()

+---+----+-------+------------+-----------+------------+----+
| id|band|guitars|        name|   hometown|        name|year|
+---+----+-------+------------+-----------+------------+----+
|  0|   0|    [0]|  Jimmy Page|     London|Led Zeppelin|1968|
|  1|   1|    [1]| Angus Young|     Sydney|       AC/DC|1973|
|  3|   3|    [3]|Kirk Hammett|Los Angeles|   Metallica|1981|
+---+----+-------+------------+-----------+------------+----+



In [35]:
# left semi joins = everything in the LEFT DF for which there is a row in the right DF for which the condition is true
# more like a filter
# equivalent SQL: select * from guitar_players WHERE EXISTS (...)
guitar_players_df.join(bands_df, join_condition, "left_semi").show()

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



In [36]:
# left anti joins = everything in the LEFT DF for which there is __NO__ row in the right DF for which the condition is true
guitar_players_df.join(bands_df, join_condition, "left_anti").show()

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



Exercises
Read the tables in the Postgres database: employees, salaries, dept_emp
1. show all employees and their max salary over time
2. show all employees who were never managers
3. for every employee, find the difference between their salary (current/latest) and 
    the max salary of their department (departments table)
"""

In [ ]:
# WORK WITH POSTGRES FROM PyCharm project - https://github.com/vadopolski/eas-017-RDD-py

def read_table(table_name):
    return spark.read. \
        format("jdbc"). \
        option("driver", driver). \
        option("url", url). \
        option("user", user). \
        option("password", password). \
        option("dbtable", "public." + table_name). \
        load()

employees_df = read_table("employees")
salaries_df = read_table("salaries")
dept_managers_df = read_table("dept_manager")
dept_emp_df = read_table("dept_emp")
departments_df = read_table("departments")
